## Observations and Insights 

Over the course of 45 days, tumor development was observed and measured.
The purpose of this study was to compare the performance of Pymaceuticals' drug of interest, Capomulin, 
versus the other treatment regimens.
I can conclude after careful, and extensive analysis that:
Capomulin was productive in reducing the tumor size over a 45 day period.
There was a substantial correlation between mouse weight and average tumor volume.
Out of the four regimens(Capomulin, Ramicane, Infubinol, and Ceftamin), Infubinol was the only drug with an outlier.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sts
import numpy as np
from scipy.stats import linregress

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
merged_df=pd.merge(mouse_metadata, study_results, on='Mouse ID')

# Display the data table for preview
merged_df.head().round(3)

FileNotFoundError: [Errno 2] No such file or directory: 'data/Mouse_metadata.csv'

In [ ]:
# Checking the number of mice.
mouse_count = merged_df['Mouse ID'].nunique()
print(f"There are {mouse_count} different mice in the study.")

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
dup_mice = merged_df.groupby(['Mouse ID', 'Timepoint'])

# ??
#dup_mice_df = merged_df.loc[:,['Mouse ID', 'Timepoint']]
#dup_mice_df

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
each_mouse_df = merged_df.drop_duplicates('Mouse ID')
each_mouse_df.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.
each_mouse_df['Mouse ID'].count()

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen:
summary_data = merged_df.groupby(['Drug Regimen'])

# mean, median, variance, standard deviation, and SEM of the tumor volume. 
summary_df = pd.DataFrame({
    "Tumor Vol Mean": summary_data['Tumor Volume (mm3)'].mean(),
    "Tumor Vol Median": summary_data['Tumor Volume (mm3)'].median(),
    "Tumor Vol Variance": summary_data['Tumor Volume (mm3)'].var(),
    "Tumor Vol Std Dev": summary_data['Tumor Volume (mm3)'].std(),
    "Tumor Vol SEM": summary_data['Tumor Volume (mm3)'].sem()
})

# Assemble the resulting series into a single summary dataframe.
summary_df.round(3).head()

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line
summary_df = summary_data.agg({'Tumor Volume (mm3)': ['mean', 'median', 'var', 'std', 'sem']})

summary_df.round(3).head()

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using PANDAS.

# Groupby drug regimen and use the unique mice count to create a new data frame
drugs = merged_df.groupby('Drug Regimen')
mice_to_drug = pd.DataFrame(drugs['Drug Regimen'].count())

# Plot the new data frame as a bar chart
mice_to_drug.plot(kind='bar', facecolor='blue', figsize=(9,4))
plt.title("Drug Regimen On Mice")
plt.xlabel("Drug Regimen")
plt.ylabel("Number of Unique Mice")
plt.tight_layout()

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using PYPLOT.

# Groupby drug regimen and use the unique mice count to create a new data frame
drugs = merged_df.groupby('Drug Regimen')
mice_to_drug = pd.DataFrame(drugs['Drug Regimen'].count())

# Set axis and tick locations
x_axis = np.arange(len(mice_to_drug))
tick_locations = [value for value in x_axis]

# Plot bar chart and set x ticks
plt.figure(figsize=(9,4))
plt.bar(x_axis, mice_to_drug['Drug Regimen'], color='b', align='center', width=.5)
plt.xticks(tick_locations, list(mice_to_drug.index), rotation="vertical")

# Set x and y limits, title, and labels
plt.xlim(-.5, len(x_axis)-.5)
plt.ylim(0, 240)
plt.title("Drug Regimen On Mice")
plt.xlabel("Drug Regimen")
plt.ylabel("Number of Unique Mice")

# Set legend(label/key)
legend_name = ["Drug Regimen"]
plt.legend(legend_name, loc='best')
plt.tight_layout()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using PANDAS
genders = merged_df.groupby('Sex')

# Create a data frame with the grouped data
gender_df = pd.DataFrame(genders['Sex'].count())
#gender_df.head()

# Use the pandas method to plot a pie chart
gender_df.plot(kind='pie', 
               subplots=True, 
               autopct='%1.1f%%', 
               title='Mice Gender Distribution Percentages', 
               colors=['lightskyblue', 'purple'], 
               shadow=True,
               legend=True)
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using PYPLOT
genders = list(gender_df.index.values)
count = gender_df['Sex']
colors= ['lightskyblue', 'purple']
explode = (0.1, 0)
plt.figure()
plt.pie(count, labels=genders, colors=colors, autopct='%1.1f%%', shadow=True)
plt.title('Mice Gender Distribution Percentages')
plt.axis('equal')
plt.ylabel('Sex')
plt.legend(loc='best')
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Grab data from only the four regimens we will be using 
regimens = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']
regimen_df = merged_df[merged_df['Drug Regimen'].isin(regimens)]

# Another way to do that:
#regimen_df = merged_df.loc[(merged_df["Drug Regimen"] == "Capomulin") | 
                          #(merged_df["Drug Regimen"] == "Ramicane") | 
                          #(merged_df["Drug Regimen"] == "Infubinol") | 
                          #(merged_df["Drug Regimen"] == "Ceftamin"), :]

# Start by getting the last (greatest) timepoint for each mouse
regimen_df = regimen_df.sort_values('Timepoint', ascending=False)
regimen_df = regimen_df.drop_duplicates(subset='Mouse ID', keep='first')
regimen_df.head().round(3)

# Another df with the index set as the drug regimen
#drug_reg_df = regimen_df.set_index('Drug Regimen')
#drug_reg_df

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
# Create empty list to fill with tumor vol data (for plotting)
tumor_data = []

# Calculate the IQR and quantitatively determine if there are any potential outliers.
# Create a for loop to loop through each regimen in our regimens list
for x in regimens:
    each_drug = regimen_df.loc[regimen_df['Drug Regimen'] == x,'Tumor Volume (mm3)'] 
    tumor_data.append(each_drug)
    quartiles = regimen_df['Tumor Volume (mm3)'].quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq

# Print lower, upper, and median quartiles, and iqr
print(f"The lower quartile of Tumor Volume (mm3) is: {lowerq}")
print(f"The upper quartile of Tumor Volume (mm3) is: {upperq}")
print(f"The interquartile range of Tumor Volume (mm3) is: {iqr}")
print(f"The the median of Tumor Volume (mm3) is: {quartiles[0.5]} ")

# Calculate and print upper and lower quartiles, to locate possible outliers
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
blue_square = dict(markerfacecolor='b', marker='s')
fig1, ax1 = plt.subplots()

# Set title, labels, limits
ax1.set_title('Tumor Volume of Mice On Each Regimen')
ax1.set_xlabel('Drug Regimen')
ax1.set_ylabel('Tumor Volume')

ax1.set_ylim(10, 80)
ax1.boxplot(tumor_data, labels=regimens, flierprops=blue_square)
plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin

cap_data = merged_df[(merged_df["Drug Regimen"] == "Capomulin") & (merged_df['Mouse ID'] == "s185")]
#cap_data.head()

# Another possible way ???
#cap_timepoint = cap_data.groupby(['Timepoint'])
#cap_timepoint.head()

time_df = cap_data.set_index('Timepoint')
#time_df.head()

x_axis = list(time_df.index.values)
tumor_vol, = plt.plot(x_axis, time_df['Tumor Volume (mm3)'], 
                      marker='o', color='blue', label='Tumor Volume')

plt.legend(handles=[tumor_vol], loc='best')
plt.title("Tumor Vol vs Time For Mouse ID s185")
plt.xlabel("Timepont")
plt.ylabel("Tumor Volume")
plt.show()

In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
regimen_weight = merged_df[(merged_df["Drug Regimen"] == "Capomulin")]
regimen_weight = regimen_weight[["Mouse ID", "Weight (g)", "Tumor Volume (mm3)"]]
each_weight = regimen_weight.groupby(["Mouse ID", "Weight (g)"])

avg_vol_df = each_weight["Tumor Volume (mm3)"].mean().to_frame()
avg_vol_df = avg_vol_df.reset_index()
#avg_vol_df.head()

plt.scatter(avg_vol_df["Weight (g)"],
            avg_vol_df["Tumor Volume (mm3)"],
            marker = 'o',
            facecolors='blue',
            edgecolors='black')

plt.title('Average Tumor Vol vs Mouse Weight')
plt.xlabel('Mouse Weight (g)')
plt.ylabel('Avg Tumor Vol (mm3)')

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
correlation = sts.pearsonr(avg_vol_df["Weight (g)"], avg_vol_df["Tumor Volume (mm3)"])
print(f"The correlation coefficient between both factors is {round(correlation[0],2)}")

In [ ]:
x_values = avg_vol_df['Weight (g)']
y_values = avg_vol_df['Tumor Volume (mm3)']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")

plt.annotate(line_eq,(18,37),fontsize=15,color="black")

plt.xlabel('Mouse Weight (g)')
plt.ylabel('Avg Tumor Vol (mm3)')

print(f"The r-squared is: {rvalue}")
print(f"The equation of the regression line is: {line_eq}")

plt.show()